In [1]:
!pip install gradio scikit-learn pandas

In [3]:
from google.colab import files
uploaded = files.upload()


Saving IMDb Movies India.csv.zip to IMDb Movies India.csv (1).zip


In [8]:
import pandas as pd
import numpy as np

# Load dataset with proper encoding
df = pd.read_csv("/content/IMDb Movies India.csv (1).zip", encoding='ISO-8859-1')

# Drop rows with missing Rating
df = df.dropna(subset=["Rating"]).copy()

# Clean 'Year'
df["Year"] = df["Year"].str.extract(r"(\d{4})").astype(float)

# Clean 'Duration'
df["Duration"] = df["Duration"].str.extract(r"(\d+)").astype(float)

# Clean 'Votes'
df["Votes"] = df["Votes"].str.replace(",", "", regex=False)
df["Votes"] = pd.to_numeric(df["Votes"], errors='coerce')

# Drop rows with missing important numeric values
df.dropna(subset=["Year", "Duration", "Votes"], inplace=True)

# Combine actors into a single string
df["Actors"] = df["Actor 1"].fillna('') + ", " + df["Actor 2"].fillna('') + ", " + df["Actor 3"].fillna('')

In [13]:
df.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Actors
1,#Gadhvi (He thought he was Gandhi),2019.0,109.0,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid,"Rasika Dugal, Vivek Ghamande, Arvind Jangid"
3,#Yaaram,2019.0,110.0,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor,"Prateik, Ishita Raj, Siddhant Kapoor"
5,...Aur Pyaar Ho Gaya,1997.0,147.0,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor,"Bobby Deol, Aishwarya Rai Bachchan, Shammi Kapoor"
6,...Yahaan,2005.0,142.0,"Drama, Romance, War",7.4,1086,Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma,"Jimmy Sheirgill, Minissha Lamba, Yashpal Sharma"
8,?: A Question Mark,2012.0,82.0,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia,"Yash Dave, Muntazir Ahmad, Kiran Bhatia"


In [9]:
from sklearn.model_selection import train_test_split

# Select input features and target variable
X = df[["Genre", "Director", "Actors", "Year", "Duration", "Votes"]]
y = df["Rating"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Fill missing text fields with empty strings
X["Genre"] = X["Genre"].fillna("")
X["Director"] = X["Director"].fillna("")
X["Actors"] = X["Actors"].fillna("")

/tmp/ipython-input-10-1336732386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Genre"] = X["Genre"].fillna("")
/tmp/ipython-input-10-1336732386.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Director"] = X["Director"].fillna("")
/tmp/ipython-input-10-1336732386.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [11]:
# Prepare features and target
X = df[["Genre", "Director", "Actors", "Year", "Duration", "Votes"]]
y = df["Rating"]

# 🔧 Fix: Fill missing text columns
X["Genre"] = X["Genre"].fillna("")
X["Director"] = X["Director"].fillna("")
X["Actors"] = X["Actors"].fillna("")

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/tmp/ipython-input-11-2253750930.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Genre"] = X["Genre"].fillna("")
/tmp/ipython-input-11-2253750930.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Director"] = X["Director"].fillna("")
/tmp/ipython-input-11-2253750930.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [12]:
import gradio as gr

# Define prediction function
def predict_rating(genre, director, actor1, actor2, actor3, year, duration, votes):
    input_df = pd.DataFrame([{
        "Genre": genre,
        "Director": director,
        "Actors": f"{actor1}, {actor2}, {actor3}",
        "Year": float(year),
        "Duration": float(duration),
        "Votes": float(votes)
    }])
    prediction = pipeline.predict(input_df)[0]
    return round(prediction, 2)

# Create Gradio interface
interface = gr.Interface(
    fn=predict_rating,
    inputs=[
        gr.Textbox(label="Genre (e.g. Drama,Comedy)"),
        gr.Textbox(label="Director"),
        gr.Textbox(label="Actor 1"),
        gr.Textbox(label="Actor 2"),
        gr.Textbox(label="Actor 3"),
        gr.Number(label="Year"),
        gr.Number(label="Duration (in minutes)"),
        gr.Number(label="Number of Votes")
    ],
    outputs="number",
    title="Movie Rating Prediction",
    description="Enter movie details to estimate its IMDb rating."
)

interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9499e00014e88a6f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
